# Building an RNN for Machine Translation
## Initial Data Work

In this file we will read in the data for the Vietnamese and Chinese to Engish corpuses, build a token2id and char2id mapping, vocabularies and data loaders

### Building an RNN for Machine Translation
Initial Data Work

In [1]:
#Import Modules
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch import optim
import pickle as pkl
import random
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from gensim.models import KeyedVectors
import io
from collections import Counter
import sacrebleu

random.seed(123)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Global Variables
PAD_IDX = 0
UNK_IDX = 1
SOS_IDX = 2
EOS_IDX = 3
BATCH_SIZE = 64

## Loading in pre-trained fasttext embeddings for the three languages
### Building loaded embeddings, token2id, id2token and ordered words for all languages

In [2]:
# Load Pre-trained Word Vectors
def load_embeddings(word2vec, word2id, embedding_dim):
    embeddings = np.zeros((len(word2id), embedding_dim))
    for word, index in word2id.items():
        try:
            embeddings[index] = word2vec[word]

        except KeyError:
            embeddings[index] = np.random.normal(scale=0.6, size=(300,))

    return embeddings


def load_vectors(fname, num_vecs=None):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))

        if num_vecs is None:
            pass
        else:
            if len(data) + 1 > num_vecs:
                break
    return data


def data_dictionary(tokens, vocab_size_limit):
    token_counter = Counter()
    for token in tokens:
        token_counter[token] += 1

    vocab, count = zip(*token_counter.most_common(vocab_size_limit))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(4, 4 + len(vocab))))
    id2token = ['<pad>', '<unk>','<sos>','<eos>'] + id2token
    token2id['<pad>'] = PAD_IDX
    token2id['<unk>'] = UNK_IDX
    token2id['<sos>'] = SOS_IDX
    token2id['<eos>'] = EOS_IDX
    return token2id, id2token

#Load Word Embeddings

path= '../pretrained_embeddings/'

en_loaded_embeddings = load_vectors(path +'wiki.en.vec',100000)
print("Total number of words embedded is {:,d}".format(len(en_loaded_embeddings)))

vi_loaded_embeddings = load_vectors(path+'wiki.vi.vec',100000)
print("Total number of words embedded is {:,d}".format(len(vi_loaded_embeddings)))

zh_loaded_embeddings = load_vectors(path+'wiki.zh.vec',100000)
print("Total number of words embedded is {:,d}".format(len(zh_loaded_embeddings)))


#Create token2Id, token2Id
en_token2id, en_id2token = data_dictionary(list(en_loaded_embeddings.keys()), 100000)
vi_token2id, vi_id2token = data_dictionary(list(vi_loaded_embeddings.keys()), 100000)
zh_token2id, zh_id2token = data_dictionary(list(zh_loaded_embeddings.keys()), 100000)


#Create Emedding Matrix
en_loaded_embeddings=load_embeddings(en_loaded_embeddings,en_token2id,300)
vi_loaded_embeddings=load_embeddings(vi_loaded_embeddings,vi_token2id,300)
zh_loaded_embeddings=load_embeddings(zh_loaded_embeddings,zh_token2id,300)

Total number of words embedded is 100,000
Total number of words embedded is 100,000
Total number of words embedded is 100,000


## Loading in training, validation and test sets

In [11]:
#Loading in the Vietnamese -> En datasets
path1= '../project_data/en-vi/'

train_vi_en = []
with open(path1 +'train.tok.en') as inputfile:
    for line in inputfile:
        train_vi_en.append(line.strip().lower().split(' '))

train_vi_vi = []
with open(path1 + 'train.tok.vi') as inputfile:
    for line in inputfile:
        train_vi_vi.append(line.strip().lower().split(' '))

val_vi_en = []
with open(path1 + 'dev.tok.en') as inputfile:
    for line in inputfile:
        val_vi_en.append(line.strip().lower().split(' '))

val_vi_vi = []
with open(path1 +'dev.tok.vi') as inputfile:
    for line in inputfile:
        val_vi_vi.append(line.strip().lower().split(' '))
        
test_vi_en = []
with open(path1 +'test.tok.en') as inputfile:
    for line in inputfile:
        test_vi_en.append(line.strip().lower().split(' '))

test_vi_vi = []
with open(path1 + 'test.tok.vi') as inputfile:
    for line in inputfile:
        test_vi_vi.append(line.strip().lower().split(' '))
        
        
        
#Loading in the Chinese -> En datasets
path2= '../project_data/en-zh-processed/'

train_zh_en = []
with open(path2 +'train.tok.en') as inputfile:
    for line in inputfile:
        train_zh_en.append(line.strip().lower().split(' '))

i=0
train_zh_zh = []
fin = io.open(path2 +'train.tok.zh', 'r', encoding='utf-8',newline= '\n', errors='ignore')
for line in fin:
    if i % 2 == 0 :
        train_zh_zh.append(line.rstrip().split(' '))
    i+=1 

val_zh_en = []
with open(path2 + 'dev.tok.en') as inputfile:
    for line in inputfile:
        val_zh_en.append(line.strip().lower().split(' '))

i=0
val_zh_zh = []
fin = io.open(path2 +'dev.tok.zh', 'r', encoding='utf-8',newline= '\n', errors='ignore')
for line in fin:
    if i % 2 == 0 :
        val_zh_zh.append(line.rstrip().split(' '))
    i+=1

test_zh_en = []
with open(path2 +'test.tok.en') as inputfile:
    for line in inputfile:
        test_zh_en.append(line.strip().lower().split(' '))

i=0
test_zh_zh = []
fin = io.open(path2 +'test.tok.zh', 'r', encoding='utf-8',newline= '\n', errors='ignore')
for line in fin:
    if i % 2 == 0 :
        test_zh_zh.append(line.rstrip().split(' '))
    i+=1

#Sanity Checking
print("Vi -> En | Training Examples: "+str(len(train_vi_en)))
print("Vi -> En | Training Examples: "+str(len(train_vi_vi)), '\n')

print("Vi -> En | Validation Examples: "+str(len(val_vi_en)))
print("Vi -> En | Validation Examples: "+str(len(val_vi_vi)), '\n')

print("Vi -> En | Testing Examples: "+str(len(test_vi_en)))
print("Vi -> En | Testing Examples: "+str(len(test_vi_vi)), '\n')

print("Zh -> En | Training Examples: "+str(len(train_zh_en)))
print("Zh -> En | Training Examples: "+str(len(train_zh_zh)), '\n')

print("Zh -> En | Validation Examples: "+str(len(val_zh_en)))
print("Zh -> En | Validation Examples: "+str(len(val_zh_zh)), '\n')

print("Zh -> En | Testing Examples: "+str(len(test_zh_en)))
print("Zh -> En | Testing Examples: "+str(len(test_zh_zh)), '\n')

Vi -> En | Training Examples: 133317
Vi -> En | Training Examples: 133317 

Vi -> En | Validation Examples: 1268
Vi -> En | Validation Examples: 1268 

Vi -> En | Testing Examples: 1553
Vi -> En | Testing Examples: 1553 

Zh -> En | Training Examples: 213377
Zh -> En | Training Examples: 213377 

Zh -> En | Validation Examples: 1261
Zh -> En | Validation Examples: 1261 

Zh -> En | Testing Examples: 1397
Zh -> En | Testing Examples: 1397 



In [4]:
#preserve original data for evaluation
train_vi_en_orig = train_vi_en
train_vi_vi_orig = train_vi_vi
val_vi_en_orig = val_vi_en
val_vi_vi_orig = val_vi_vi
test_vi_en_orig = test_vi_en
test_vi_vi_orig = test_vi_vi

train_zh_en_orig = train_zh_en
train_zh_zh_orig = train_zh_zh
val_zh_en_orig = val_zh_en
val_zh_zh_orig = val_zh_zh
test_zh_en_orig = test_zh_en
test_zh_zh_orig = test_zh_zh

### Encoding our data

In [12]:
VI_EN_MAX_LENGTH = int(np.percentile([len(sentence) for sentence in train_vi_en+train_vi_vi], 90))+1
ZH_EN_MAX_LENGTH = int(np.percentile([len(sentence) for sentence in train_zh_en+train_zh_zh], 90))+1

In [6]:
def encoding_tokens(sentence, language, translator):
    if language== 'English':
        token2id = en_token2id
    elif language== 'Vietnamese':
        token2id = vi_token2id
    elif language== 'Chinese':
        token2id = zh_token2id
    tokens = [token2id[token] if token in token2id else UNK_IDX for token in sentence]
    if translator == 'vi':
        max_len = VI_EN_MAX_LENGTH-1
    elif translator == 'zh':
        max_len = ZH_EN_MAX_LENGTH-1
    tokens=tokens[:max_len]
    return tokens

def encoding_dataset(dataset, language, translator):
    data = [encoding_tokens(tokens, language, translator) for tokens in dataset] 
    return data

In [13]:
train_vi_en = encoding_dataset(train_vi_en, 'English', 'vi')
train_vi_vi = encoding_dataset(train_vi_vi, 'Vietnamese', 'vi')
test_vi_en = encoding_dataset(test_vi_en, 'English', 'vi')
test_vi_vi = encoding_dataset(test_vi_vi, 'Vietnamese', 'vi')
val_vi_en = encoding_dataset(val_vi_en, 'English', 'vi')
val_vi_vi = encoding_dataset(val_vi_vi, 'Vietnamese', 'vi')

train_zh_en = encoding_dataset(train_zh_en, 'English', 'zh')
train_zh_zh = encoding_dataset(train_zh_zh, 'Chinese', 'zh')
test_zh_en = encoding_dataset(test_zh_en, 'English', 'zh')
test_zh_zh = encoding_dataset(test_zh_zh, 'Chinese', 'zh')
val_zh_en = encoding_dataset(val_zh_en, 'English', 'zh')
val_zh_zh = encoding_dataset(val_zh_zh, 'Chinese', 'zh')

## Building Data Loaders

In [8]:
class translationDataset(Dataset):
    def __init__(self, data_list, target_list):
        self.data_list=data_list
        self.target_list=target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        data = self.data_list[key][:MAX_SAMPLE_LENGTH]
        label = self.target_list[key][:MAX_SAMPLE_LENGTH]
        return [data, len(data), label, len(label)]

def translation_collate_func(batch):
    data_list = []
    label_list = []
    for datum in batch:
        padded_data = np.pad(np.array(datum[0]+[EOS_IDX]), 
                                pad_width=((0,MAX_SAMPLE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_data)
        padded_label = np.pad(np.array(datum[2]+[EOS_IDX]), 
                                pad_width=((0,MAX_SAMPLE_LENGTH-datum[3])), 
                                mode="constant", constant_values=0)
        label_list.append(padded_label)
    return [torch.from_numpy(np.array(data_list)), torch.from_numpy(np.array(label_list))]

In [14]:
# VI -> EN | dataloaders
MAX_SAMPLE_LENGTH = VI_EN_MAX_LENGTH

vi_en_train_dataset = translationDataset(train_vi_vi, train_vi_en)
vi_en_train_loader = torch.utils.data.DataLoader(dataset=vi_en_train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

vi_en_val_dataset = translationDataset(val_vi_vi, val_vi_en)
vi_en_val_loader = torch.utils.data.DataLoader(dataset=vi_en_val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

vi_en_test_dataset = translationDataset(test_vi_vi, test_vi_en)
vi_en_test_loader = torch.utils.data.DataLoader(dataset=vi_en_test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

In [15]:
# ZH -> EN | dataloaders
MAX_SAMPLE_LENGTH = ZH_EN_MAX_LENGTH

zh_en_train_dataset = translationDataset(train_zh_zh, train_zh_en)
zh_en_train_loader = torch.utils.data.DataLoader(dataset=zh_en_train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

zh_en_val_dataset = translationDataset(val_zh_zh, val_zh_en)
zh_en_val_loader = torch.utils.data.DataLoader(dataset=zh_en_val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

zh_en_test_dataset = translationDataset(test_zh_zh, test_zh_en)
zh_en_test_loader = torch.utils.data.DataLoader(dataset=zh_en_test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=translation_collate_func,
                                           shuffle=True)

## Building the RNN model

In [16]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def showPlot(points, string):
    plt.figure()
    fig, ax = plt.subplots()
    plt.plot(points)
    plt.title(string)
    plt.savefig((string+'.png'), dpi=300)

In [127]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, language, drop_rate=0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.language = language
        if language == 'Vietnamese':
             self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vi_loaded_embeddings), freeze=True)
        elif language == 'Chinese':
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vi_loaded_embeddings), freeze=True)
        self.gru = nn.GRU(300, hidden_size)
        self.dropout = nn.Dropout(drop_rate)

    def forward(self, input):
        embedded = self.embedding(input)
        output = self.dropout(embedded)
        output, hidden = self.gru(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, drop_rate=0):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(en_loaded_embeddings), freeze=True)
        self.gru = nn.GRU(hidden_size + 300, hidden_size)
        self.out = nn.Linear(hidden_size, len(en_token2id))
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(p=drop_rate)

    def forward(self, input, hidden, enc_output):
        input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input)).unsqueeze(0)
        embedded_concat = torch.cat((embedded, enc_output), dim=2)
        output, hidden = self.gru(embedded_concat, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_tensor = input_tensor.transpose(0,1)
    target_tensor = target_tensor.transpose(0,1)
    
    max_length = input_tensor.size(0)
    batch_size = input_tensor.size(1)
    vocab_size = len(en_token2id)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    
    encoder_output, encoder_hidden = encoder(input_tensor)
    encoder_outputs = encoder_output[0,0]

    decoder_input = input_tensor[0,:]
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
               decoder_input, decoder_hidden, encoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
               decoder_input, decoder_hidden, encoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input[di].item() == EOS_IDX:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / max_length

def trainIters(loader, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    language = encoder.language
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        for i, (data, labels) in enumerate(loader):
            input_tensor = data
            target_tensor = labels
    
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses, language)
    return plot_losses

In [128]:
#Training Model
teacher_forcing_ratio = 0.5
hidden_size = 256
batch_size = 128
encoder1 = EncoderRNN(hidden_size, 'Vietnamese', drop_rate = 0.1).to(device)
decoder1 = DecoderRNN(hidden_size).to(device)
trainIters(vi_en_val_loader, encoder1, decoder1, n_iters=10, print_every=1,plot_every=1)

dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 

dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 64, 300])
dec enc_output torch.Size([1, 64, 256])
dec input torch.Size([64])
dec embedded torch.Size([1, 

KeyboardInterrupt: 

In [258]:
#beam data structure
        
class Beam_Node(object):
    def __init__(self, hiddenstate, previousNode, wordId, Prob, length):
        self.hidden = hiddenstate
        self.prevNode = previousNode
        self.wordId = wordId
        self.prob = Prob
        self.length = length


In [20]:
def indexesFromSentence(lang, sentence):
    if lang == 'Vietnamese':
        out = []
        for word in sentence:
            if word in vi_token2id:
                out.append(vi_token2id[word])
            else:
                out.append(UNK_IDX)
    elif lang == 'English':
        out = []
        for word in sentence:
            if word in en_token2id:
                out.append(en_token2id[word])
            else:
                out.append(UNK_IDX)
    elif lang == 'Chinese':
        out = []
        for word in sentence:
            if word in zh_token2id:
                out.append(zh_token2id[word])
            else:
                out.append(UNK_IDX)
    return out

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_IDX)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

In [287]:
from queue import PriorityQueue

def evaluate_beam(encoder, decoder, sentence, input_lang, beam_size=3):
    with torch.no_grad():
        beam_size = beam_size + 1 #add extra to account for padding
        input_tensor = tensorFromSentence(input_lang, sentence)
        max_length = input_tensor.size(0)
        batch_size = max_length
        input_tensor = input_tensor.transpose(0,1)
        input_length = input_tensor.size()[0]
        
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        encoder_output, encoder_hidden = encoder(input_tensor)
        encoder_outputs = encoder_output[0,0]

        decoder_input = input_tensor[0,:]
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoded_sentence = []

        beam_storage = []
        for i in range(beam_size-1):
            beam_storage.append([])
            
        for di in range(max_length):
            beams = []
            for x in range(beam_size**2):
                beams.append([])
                
            #decoder_hidden = decoder_hidden[:, di, :].unsqueeze(0)
            #encoder_output = encoder_outputs[:, di, :].unsqueeze(1)

            # starting node -  self, hidden, prevNode, wordId, prob, length
            node = Beam_Node(decoder_hidden, None, torch.LongTensor([[SOS_IDX]]), 0, 1)
            nodes = PriorityQueue()
            nodes.put((-node.prob, node))
            pqueue_size = 1
            
            # start beam search
            while True:
                # give up when sentence length too large
                if pqueue_size > 2000: 
                    print("ended early")
                    break
                    
                score, node = nodes.get()
                decoder_input = node.wordId
                decoder_hidden = node.hidden
                print(decoder_input.size())
                             
                if node.wordId[0].item() == EOS_IDX and node.prevNode != None:
                    decoded_words.append((score, node))
                    break
                
                decoder_output, decoder_hidden = decoder(decoder_input[0], decoder_hidden, encoder_output)
                prob, idx = torch.topk(decoder_output, beam_size)
                nextnodes = []
                
                for next_n in range(beam_size):
                    decoded = idx[0][next_n].view(1, -1)
                    p = prob[0][next_n].item()

                    new_node = Beam_Node(decoder_hidden, node, decoded, node.prob + p, node.length + 1)
                    score = new_node.prob
                    nextnodes.append((-score, new_node))
                                      
                # add to queue
                for i in range(len(nextnodes)):
                    score, nn = nextnodes[i]
                    nodes.put((score, nn))
                    # increase priority queue size
                    pqueue_size += len(nextnodes) - 1

            # choose best paths and trace back
            if len(decoded_words) == 0:
                decoded_words = [nodes.get() for _ in range(topk)]    


            sentence = []
            for score, n in sorted(decoded_words, key=operator.itemgetter(0)):
                words = []
                words.append(en_id2token[n.wordid])
                # get prev nodes
                while n.prevNode != None:
                    n = n.prevNode
                    words.append(n.wordid)

                words = words[::-1]
                sentence.append(words)

            decoded_sentence.append(sentence)   
                    
        return decoded_sentence              
  

In [288]:
from random import randint
index = randint(0, len(train_vi_en_orig))
sentence1 = train_vi_vi_orig[index] 
sentence2 = train_vi_en_orig[index]            

print('>', sentence1)
print('=', sentence2)
output_words = evaluate_beam(encoder1, decoder1, sentence1, "Vietnamese")
output_sentence = ' '.join(output_words)
print('<', output_sentence)
print('')

> ['khi', 'sự_nghiệp', 'của', 'mình', 'phát_triển', ',', 'david', 'byrne', 'từ', 'chơi', 'nhạc', 'trong', 'quán', 'cbgb', 'để', 'đến', 'được', 'biểu_diễn', 'ở', 'carnegie', 'hall', '.', 'ông', 'đặt', 'ra', 'một', 'câu', 'hỏi', ':', 'địa_điểm', 'tổ_chức', 'có', 'làm_nên', 'âm_nhạc', '?', 'từ', 'tiếng', 'trống', 'ngoài_trời', 'cho', 'tới', 'những', 'buổi', 'nhạc_kịch', 'wagner', 'và', 'rồi', 'là', 'sân_khấu', 'nhạc', 'rock', ',', 'ông', 'tìm_hiểu', 'cách', 'không_gian', 'đã', 'giúp', 'thúc_đẩy', 'sáng_tạo', 'âm_nhạc', '.']
= ['as', 'his', 'career', 'grew', ',', 'david', 'byrne', 'went', 'from', 'playing', 'cbgb', 'to', 'carnegie', 'hall', '.', 'he', 'asks', ':', 'does', 'the', 'venue', 'make', 'the', 'music', '?', 'from', 'outdoor', 'drumming', 'to', 'wagnerian', 'operas', 'to', 'arena', 'rock', ',', 'he', 'explores', 'how', 'context', 'has', 'pushed', 'musical', 'innovation', '.']
torch.Size([1, 1])
dec input torch.Size([1])
dec embedded torch.Size([1, 1, 300])
dec enc_output torch.Size

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 2. Got 1 and 63 in dimension 1 at /Users/soumith/miniconda2/conda-bld/pytorch_1532624435833/work/aten/src/TH/generic/THTensorMath.cpp:3616

In [161]:
from random import randint
def evaluateRandomly(encoder, decoder, language, n=10):
    for i in range(n):
        if language == 'Vietnamese':
            index = randint(0, len(train_vi_en_orig))
            sentence1 = train_vi_vi_orig[index] 
            sentence2 = train_vi_en_orig[index]
        elif language == 'Chinese':
            index = randint(0, len(train_zh_en_orig))
            sentence1 = train_zh_zh_orig[index]
            sentence2 = train_zh_en_orig[index]
        
        print('>', sentence1)
        print('=', sentence2)
        output_words = evaluate(encoder, decoder, sentence1, language)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [18]:
MAX_LENGTH = VI_EN_MAX_LENGTH
evaluateRandomly(encoder1, decoder1, "Vietnamese")

> ['chong_chóng', 'quay', 'là', 'một', '"', 'đặc_tính', 'hợp', 'trội', '"', 'xảy_ra', 'do', 'tương_tác', 'giữa', 'những', 'chú', 'chó', 'con', 'mà', 'quy_luật', 'duy_nhất', 'là', 'cố_gắng', 'duy_trì', 'sự', 'tiếp_cận', 'của', 'chúng', 'với', 'tô', 'sữa', '.', 'và', 'do', 'đó', ',', 'đẩy', 'chúng', 'đi', 'theo', 'một', 'hướng', 'ngẫu_nhiên', '.']
= ['the', 'pinwheel', 'is', 'an', 'emergent', 'property', 'of', 'the', 'interactions', 'between', 'puppies', 'whose', 'only', 'rule', 'is', 'to', 'try', 'to', 'keep', 'access', 'to', 'the', 'milk', 'and', 'therefore', 'to', 'push', 'in', 'a', 'random', 'direction', '.']
< <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

> ['sally', 'tiếp_lời', ':', '"', 'cho', 'sự_kiện', 'mùa_hè', 'ở', 'trường', 'nyu.', '"']
= ['sally', ':', '&quot;

In [19]:
#calculate BLEU score on corpus level
def BLEU(encoder, decoder, language):
    hypotheses = ""
    targets = ""
    if language == 'Vietnamese':
        inputs = val_vi_vi_orig
        labels = val_vi_en_orig
    else:
        inputs = val_zh_zh_orig
        labels = val_zh_zh_orig
        
    for sentence in inputs:
        output = evaluate(encoder, decoder, sentence, language)
        for word in output:
            hypotheses = hypotheses + " " + word

    #targets
    for sentence in labels:
        for word in sentence:
            if word in en_id2token:
                targets = targets + " " + word
            else:
                targets = targets + " " + '<unk>'

    score = sacrebleu.corpus_bleu(hypotheses, targets)
    print(score)

In [20]:
BLEU(encoder1, decoder1, 'Vietnamese')

BLEU(score=0.020322293857056525, counts=[2273, 50, 1, 0], totals=[75975, 75974, 75973, 75972], precisions=[2.9917736097400462, 0.06581198831179087, 0.0013162570913350796, 0.0006581372084452166], bp=1.0, sys_len=75975, ref_len=29000)
